Cluster the pickup time of the day into 30-minute intervals (e.g. from 02:00 to 02:30)
For each interval, determine the average number of passengers and the average fare amount.
For each payment type and each interval, determine the average fare amount
For each payment type, determine the interval when the average fare amount is maximum
For each payment type, determine the interval when the overall ratio between the tip and the fare amounts is maximum
Find the location with the highest average fare amount
Build a new dataframe (called common) where, for each pickup location we keep all trips to the 5 most common destinations (i.e. each pickup location can have different common destinations).
On the common dataframe, for each payment type and each interval, determine the average fare amount
Compute the difference of the average fare amount computed in the previous point with those computed at point 9.
Compute the ratio between the differences computed in the previous point and those computed in point 9. Note: you have to compute a ratio for each pair (payment type, interval).
Build chains of trips. Two trips are consecutive in a chain if (a) they have the same VendorID, (b) the pickup location of the second trip is also the dropoff location of the first trip, (c) the pickup time of the second trip is after the dropoff time of the first trip, and (d) the pickup time of the second trip is at most 2 minutes later than the dropoff time of the first trip.
Hint: Add a column chain to the dataset. A chain can have more than two trips.

In [1]:
#Importing Pandas Library
import pandas as pd

### 1. Extract all trips with trip_distance larger than 50

In [4]:
# Loading data into Dataframe
df = pd.read_csv('data.csv', low_memory=False)

# The condition which is asked
filtered_trips = df[df['trip_distance']>50]
print(filtered_trips)

         VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
23842         2.0  2020-01-01 01:53:07   2020-01-01 03:54:41              1.0   
39013         2.0  2020-01-01 02:05:07   2020-01-01 03:03:10              1.0   
41620         1.0  2020-01-01 03:05:54   2020-01-01 04:16:26              1.0   
58262         2.0  2020-01-01 05:36:12   2020-01-01 06:40:06              1.0   
63024         2.0  2020-01-01 07:40:30   2020-01-01 08:40:01              1.0   
...           ...                  ...                   ...              ...   
6326169       2.0  2020-01-31 22:47:26   2020-01-31 23:49:14              1.0   
6331181       2.0  2020-01-31 23:45:36   2020-02-01 01:00:25              5.0   
6333801       2.0  2020-01-31 23:24:16   2020-02-01 01:32:56              1.0   
6397132       NaN  2020-01-28 11:54:00   2020-01-28 19:35:00              NaN   
6398446       NaN  2020-01-28 07:37:00   2020-01-28 08:42:00              NaN   

         trip_distance  Rat

### 2. Extract all trips where payment_type is missing

In [7]:
# Filter rows where 'payment_type' is missing (NaN) and store them in a new DataFrame
trips_with_missing_payment_type = df[df['payment_type'].isnull()]

# Print the DataFrame containing trips with missing 'payment_type'
print(trips_with_missing_payment_type)

         VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
6339567       NaN  2020-01-01 08:51:00   2020-01-01 09:19:00              NaN   
6339568       NaN  2020-01-01 08:38:43   2020-01-01 08:51:08              NaN   
6339569       NaN  2020-01-01 08:27:00   2020-01-01 08:32:00              NaN   
6339570       NaN  2020-01-01 08:46:00   2020-01-01 08:57:00              NaN   
6339571       NaN  2020-01-01 08:21:00   2020-01-01 08:38:00              NaN   
...           ...                  ...                   ...              ...   
6405003       NaN  2020-01-31 22:51:00   2020-01-31 23:22:00              NaN   
6405004       NaN  2020-01-31 22:10:00   2020-01-31 23:26:00              NaN   
6405005       NaN  2020-01-31 22:50:07   2020-01-31 23:17:57              NaN   
6405006       NaN  2020-01-31 22:25:53   2020-01-31 22:48:32              NaN   
6405007       NaN  2020-01-31 22:44:00   2020-01-31 23:06:00              NaN   

         trip_distance  Rat

### 3. For each (PULocationID, DOLocationID) pair, determine the number of trips

In [10]:
# Group the DataFrame by 'PULocationID' and 'DOLocationID' and count the number of occurrences (i.e., the number of trips) for 
# each combination of these two columns.
num_of_trips = df.groupby(['PULocationID','DOLocationID']).size()

# Print the resulting Series with the number of trips for each (PULocationID, DOLocationID) pair
print(num_of_trips)

PULocationID  DOLocationID
1             1                638
              50                 1
              68                 1
              138                2
              140                1
                              ... 
265           259                2
              261                1
              263                4
              264              317
              265             2508
Length: 31277, dtype: int64


### 4. Save all rows with missing VendorID, passenger_count, store_and_fwd_flag, payment_type in a new dataframe called bad, and remove those rows from the original dataframe.

In [13]:
# Filter rows with missing values in any of the specified columns
bad = df[df[['VendorID','passenger_count','store_and_fwd_flag','payment_type']].isnull().any(axis=1)]

# Print the DataFrame containing rows with missing values in any of the selected columns
print(bad)

         VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
6339567       NaN  2020-01-01 08:51:00   2020-01-01 09:19:00              NaN   
6339568       NaN  2020-01-01 08:38:43   2020-01-01 08:51:08              NaN   
6339569       NaN  2020-01-01 08:27:00   2020-01-01 08:32:00              NaN   
6339570       NaN  2020-01-01 08:46:00   2020-01-01 08:57:00              NaN   
6339571       NaN  2020-01-01 08:21:00   2020-01-01 08:38:00              NaN   
...           ...                  ...                   ...              ...   
6405003       NaN  2020-01-31 22:51:00   2020-01-31 23:22:00              NaN   
6405004       NaN  2020-01-31 22:10:00   2020-01-31 23:26:00              NaN   
6405005       NaN  2020-01-31 22:50:07   2020-01-31 23:17:57              NaN   
6405006       NaN  2020-01-31 22:25:53   2020-01-31 22:48:32              NaN   
6405007       NaN  2020-01-31 22:44:00   2020-01-31 23:06:00              NaN   

         trip_distance  Rat

In [15]:
# Dropping all the data that has the same index as bad df and storing them into new df 
cleaned_df =df.drop(bad.index)

# Printing new df with no missing values
cleaned_df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.20,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.20,1.0,N,239,238,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.60,1.0,N,238,238,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.80,1.0,N,238,151,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.00,1.0,N,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339562,2.0,2020-01-31 23:38:07,2020-01-31 23:52:21,1.0,2.10,1.0,N,163,246,1.0,11.0,0.5,0.5,2.96,0.0,0.3,17.76,2.5
6339563,2.0,2020-01-31 23:00:18,2020-01-31 23:19:18,1.0,2.13,1.0,N,164,79,1.0,13.0,0.5,0.5,3.36,0.0,0.3,20.16,2.5
6339564,2.0,2020-01-31 23:24:22,2020-01-31 23:40:39,1.0,2.55,1.0,N,79,68,1.0,12.5,0.5,0.5,3.26,0.0,0.3,19.56,2.5
6339565,2.0,2020-01-31 23:44:22,2020-01-31 23:54:00,1.0,1.61,1.0,N,100,142,2.0,8.5,0.5,0.5,0.00,0.0,0.3,12.30,2.5


### 5. Add a duration column storing how long each trip has taken (use tpep_pickup_datetime, tpep_dropoff_datetime)

In [28]:
# Convert string columns to datetime
cleaned_df['tpep_pickup_datetime'] = pd.to_datetime(cleaned_df['tpep_pickup_datetime'])
cleaned_df['tpep_dropoff_datetime'] = pd.to_datetime(cleaned_df['tpep_dropoff_datetime'])

# Calculate duration as the difference between the two columns
cleaned_df['duration'] = (cleaned_df['tpep_dropoff_datetime'] - cleaned_df['tpep_pickup_datetime']).dt.total_seconds()/60  # duration in minutes
cleaned_df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,duration
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.20,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5,4.800000
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.20,1.0,N,239,238,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5,7.416667
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.60,1.0,N,238,238,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5,6.183333
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.80,1.0,N,238,151,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0,4.850000
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.00,1.0,N,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0,2.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339562,2.0,2020-01-31 23:38:07,2020-01-31 23:52:21,1.0,2.10,1.0,N,163,246,1.0,11.0,0.5,0.5,2.96,0.0,0.3,17.76,2.5,14.233333
6339563,2.0,2020-01-31 23:00:18,2020-01-31 23:19:18,1.0,2.13,1.0,N,164,79,1.0,13.0,0.5,0.5,3.36,0.0,0.3,20.16,2.5,19.000000
6339564,2.0,2020-01-31 23:24:22,2020-01-31 23:40:39,1.0,2.55,1.0,N,79,68,1.0,12.5,0.5,0.5,3.26,0.0,0.3,19.56,2.5,16.283333
6339565,2.0,2020-01-31 23:44:22,2020-01-31 23:54:00,1.0,1.61,1.0,N,100,142,2.0,8.5,0.5,0.5,0.00,0.0,0.3,12.30,2.5,9.633333


### 6. For each pickup location, determine how many trips have started there.

In [30]:
trips_distinct_loc = cleaned_df.groupby('PULocationID').size().reset_index(name='trip_count')
print(trips_distinct_loc)

     PULocationID  trip_count
0               1         753
1               2           3
2               3          70
3               4        9902
4               5          39
..            ...         ...
255           261       34229
256           262       85591
257           263      123997
258           264       43779
259           265        3090

[260 rows x 2 columns]


### 7. Cluster the pickup time of the day into 30-minute intervals (e.g. from 02:00 to 02:30)

In [42]:
cleaned_df['pickup_time_interval'] = (cleaned_df['tpep_pickup_datetime'].dt.hour * 60 + cleaned_df['tpep_pickup_datetime'].dt.minute) // 30

In [44]:
cleaned_df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,duration,pickup_time_interval
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.20,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5,4.800000,0
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.20,1.0,N,239,238,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5,7.416667,1
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.60,1.0,N,238,238,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5,6.183333,1
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.80,1.0,N,238,151,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0,4.850000,1
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.00,1.0,N,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0,2.300000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339562,2.0,2020-01-31 23:38:07,2020-01-31 23:52:21,1.0,2.10,1.0,N,163,246,1.0,11.0,0.5,0.5,2.96,0.0,0.3,17.76,2.5,14.233333,47
6339563,2.0,2020-01-31 23:00:18,2020-01-31 23:19:18,1.0,2.13,1.0,N,164,79,1.0,13.0,0.5,0.5,3.36,0.0,0.3,20.16,2.5,19.000000,46
6339564,2.0,2020-01-31 23:24:22,2020-01-31 23:40:39,1.0,2.55,1.0,N,79,68,1.0,12.5,0.5,0.5,3.26,0.0,0.3,19.56,2.5,16.283333,46
6339565,2.0,2020-01-31 23:44:22,2020-01-31 23:54:00,1.0,1.61,1.0,N,100,142,2.0,8.5,0.5,0.5,0.00,0.0,0.3,12.30,2.5,9.633333,47
